In [1]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
size = [224,224,3]

train_path = "Datasets/train"
test_path = "Datasets/test"

In [3]:
resnet = ResNet50(include_top=False, weights='imagenet',input_shape=size)

In [4]:
for layer in resnet.layers:
    layer.trainable = False

In [5]:
folders = glob('Datasets/train/*')

In [6]:
folders

['Datasets/train\\audi',
 'Datasets/train\\lamborghini',
 'Datasets/train\\mercedes']

In [7]:
x = Flatten()(resnet.output)

In [8]:
prediction = Dense(len(folders) , activation='softmax')(x)

In [9]:
model = Model(inputs=resnet.input , outputs=prediction)

In [10]:
model

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [12]:
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [13]:
train_datagen = ImageDataGenerator(shear_range=0.2,
    zoom_range=0.2,rescale=1/255)

In [14]:
test_datagen = ImageDataGenerator(rescale=1/255)

In [15]:
training_set = train_datagen.flow_from_directory("Datasets/train" , batch_size=32,class_mode='categorical',target_size = (224,224) )

Found 64 images belonging to 3 classes.


In [16]:
test_set = test_datagen.flow_from_directory("Datasets/test" , batch_size=32,class_mode='categorical',target_size = (224,224) )

Found 58 images belonging to 3 classes.


In [21]:
r = model.fit_generator(training_set, 
                        epochs=20,
                        steps_per_epoch=len(training_set),
                        validation_steps=len(test_set),
                        validation_data=test_set)

C:\Users\ssc\AppData\Local\Temp\ipykernel_1068\1367657369.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(training_set,


Epoch 1/20
2/2 [==============================] - 15s 10s/step - loss: 3.0818 - accuracy: 0.5469 - val_loss: 4.5817 - val_accuracy: 0.3448
Epoch 2/20
2/2 [==============================] - 15s 11s/step - loss: 2.8768 - accuracy: 0.5312 - val_loss: 8.1823 - val_accuracy: 0.1552
Epoch 3/20
2/2 [==============================] - 15s 11s/step - loss: 6.8778 - accuracy: 0.4219 - val_loss: 2.5213 - val_accuracy: 0.3966
Epoch 4/20
2/2 [==============================] - 17s 12s/step - loss: 2.8961 - accuracy: 0.5312 - val_loss: 2.4981 - val_accuracy: 0.6034
Epoch 5/20
2/2 [==============================] - 17s 12s/step - loss: 2.7253 - accuracy: 0.6562 - val_loss: 2.2895 - val_accuracy: 0.3276
Epoch 6/20
2/2 [==============================] - 18s 13s/step - loss: 3.8519 - accuracy: 0.5156 - val_loss: 1.5874 - val_accuracy: 0.6552
Epoch 7/20
2/2 [==============================] - 27s 19s/step - loss: 2.2038 - accuracy: 0.6094 - val_loss: 4.9357 - val_accuracy: 0.5345
Epoch 8/20
2/2 [===========

In [22]:
r.history

{'loss': [3.0818283557891846,
  2.876777410507202,
  6.877763271331787,
  2.896108865737915,
  2.7252609729766846,
  3.851851463317871,
  2.203831672668457,
  2.5719923973083496,
  3.45668363571167,
  2.172109603881836,
  2.5029890537261963,
  2.099818468093872,
  3.5065994262695312,
  4.337717533111572,
  2.4569966793060303,
  3.265578508377075,
  5.317690849304199,
  3.613872528076172,
  3.4286725521087646,
  3.573988914489746],
 'accuracy': [0.546875,
  0.53125,
  0.421875,
  0.53125,
  0.65625,
  0.515625,
  0.609375,
  0.609375,
  0.59375,
  0.703125,
  0.5625,
  0.71875,
  0.609375,
  0.5625,
  0.640625,
  0.65625,
  0.5,
  0.640625,
  0.609375,
  0.515625],
 'val_loss': [4.581651210784912,
  8.18234920501709,
  2.521300792694092,
  2.498089075088501,
  2.289543390274048,
  1.587355136871338,
  4.935746669769287,
  9.24686336517334,
  4.02777099609375,
  2.1247050762176514,
  5.553137302398682,
  2.6422760486602783,
  5.348113536834717,
  2.1162142753601074,
  2.36456036567688,
 

In [23]:
model.save('resnet.h5')

In [28]:
y_pred = model.predict(test_set)

2/2 [==============================] - 7s 3s/step


In [30]:
import numpy as np

In [31]:
y_pred = np.argmax(y_pred, axis=1)

In [32]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [33]:
from tensorflow.keras.models import load_model

In [34]:
from tensorflow.keras.preprocessing import image

In [35]:
model = load_model('resnet.h5')

In [61]:
img = image.load_img('Datasets/test/audi/22.jpg',target_size=(224,224))

In [62]:
x=image.img_to_array(img)

In [63]:
x.shape

(224, 224, 3)

In [64]:
x=x/255

In [66]:
x=np.expand_dims(x,axis=0)
x = preprocess_input(x)


In [69]:
result = np.argmax(model.predict(x),axis=1)

1/1 [==============================] - 0s 329ms/step


In [70]:
result

array([1], dtype=int64)